In [43]:
import pandas as pd
import numpy as np
import os
import matplotlib as plt

In [2]:
df1 = pd.read_csv ("//ad.univ-lille.fr/Etudiants/Homedir3/287058/Documents/M2/MLOPS/MLOPS-TD1-23/donnéestd/restaurant_1_week_002.csv")

In [4]:
df1.head()

,Order Number,Order Date,Item Name,Quantity,Product Price,Total products
0,1388,2015-01-10 20:12:00,Onion Chutney,1,0.50,6
1,1388,2015-01-10 20:12:00,Mint Sauce,1,0.50,6
2,1388,2015-01-10 20:12:00,Lime Pickle,1,0.50,6
3,1388,2015-01-10 20:12:00,Paratha,1,2.95,6
4,1387,2015-01-10 16:55:00,Onion Bhaji,1,3.95,1


In [5]:
df2 = pd.read_csv ("//ad.univ-lille.fr/Etudiants/Homedir3/287058/Documents/M2/MLOPS/MLOPS-TD1-23/donnéestd/restaurant_2_week_235.csv")

In [6]:
df2.head()

,Order ID,Order Date,Item Name,Quantity,Product Price,Total products
0,25061,2019-07-07 20:52:00,Mango Chutney,2,0.5,8
1,25061,2019-07-07 20:52:00,Red Sauce,1,0.5,8
2,25056,2019-07-07 19:40:00,Mint Sauce,1,0.5,7
3,25055,2019-07-07 19:36:00,Red Sauce,1,0.5,8
4,25055,2019-07-07 19:36:00,Onion Chutney,1,0.5,8


In [35]:
def extract(
    data_dir: str, 
    prefix: str, 
    start_week: int, 
    end_week: int) -> pd.DataFrame:
    """ Function to load and concat batch data.
    
    Parameters:
    -----------
    data_dir: str
        directory where data is located
    prefix: str
        restaurant name (ex. restaurant_1)
    start_week: int
        start week to load
    end_week: int
        end week to load
    
    Return: 
    df: pd.Dataframe
        Dataframe load
    """
    df = pd.DataFrame()
    for i in range(start_week, end_week+1):
        file_path = os.path.join(data_dir, "donnéestd", f'{prefix}_week_{i}.csv')
        if os.path.isfile(file_path):
            batch = pd.read_csv(file_path)
            df = pd.concat([df, batch], sort=True)
    return df

In [36]:
df_rest1 = extract(data_dir = '//ad.univ-lille.fr/Etudiants/Homedir3/287058/Documents/M2/MLOPS/MLOPS-TD1-23', prefix = "restaurant_1", start_week = 108, end_week = 110)
df_rest2= extract(data_dir = '//ad.univ-lille.fr/Etudiants/Homedir3/287058/Documents/M2/MLOPS/MLOPS-TD1-23', prefix = "restaurant_2", start_week = 108, end_week = 110)

In [37]:
def clean(df: pd.DataFrame) -> pd.DataFrame:
    df.columns = df.columns.str.lower().str.replace(" ", "_")
    df['order_date'] = pd.to_datetime(df['order_date'])
    df = df.rename(columns={'order_number': 'order_id'})
    df = df.sort_values('order_date')
    df['total_product_price'] = df['quantity'] * df['product_price']
    df['cash_in'] = df.groupby('order_id')['total_product_price'].transform(np.sum)
    df = df.drop(columns=['item_name', 'quantity', 'product_price'
                      'total_products', 'total_product_price'],
             errors='ignore'
             )
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)
    return df

In [38]:
df_rest1_clean = clean(df_rest1)
df_rest2_clean = clean(df_rest2) 

In [40]:
def merge(df1, df2):
    df = pd.concat([df1, df2], sort=True)
    df = df.drop(columns='order_id')
    df = df.sort_values('order_date')
    df = df.reset_index(drop=True)
    return df

In [41]:
df = merge(df_rest1_clean, df_rest1_clean)

In [42]:
df.resample('1H', on='order_date').sum().reset_index()

,order_date,cash_in,product_price,total_products
0,2017-01-23 16:00:00,357.2,55.6,56
1,2017-01-23 17:00:00,348.0,56.4,72
2,2017-01-23 18:00:00,263.0,52.6,50
3,2017-01-23 19:00:00,1110.2,166.9,260
4,2017-01-23 20:00:00,0.0,0.0,0
...,...,...,...,...
481,2017-02-12 17:00:00,905.9,113.8,196
482,2017-02-12 18:00:00,1916.9,243.2,426
483,2017-02-12 19:00:00,2325.8,247.5,430
484,2017-02-12 20:00:00,978.2,135.3,214


In [24]:
plt.plot(, df.cash_in)